In [0]:
import pandas as pd

In [0]:
usecols = ['AAGE','ADTIND','ADTOCC','SEOTR','WKSWORK', 'PTOTVAL']

In [0]:
train_url = 'https://github.com/PacktWorkshops/The-TensorFlow-Workshop/blob/master/Chapter06/dataset/census-income-train.csv?raw=true'

In [0]:
train_data = pd.read_csv(train_url, usecols=usecols)
train_data.head()

,AAGE,ADTIND,ADTOCC,SEOTR,WKSWORK,PTOTVAL,ACLSWKR_ Federal government,ACLSWKR_ Local government,ACLSWKR_ Never worked,ACLSWKR_ Not in universe,ACLSWKR_ Private,ACLSWKR_ Self-employed-incorporated,ACLSWKR_ Self-employed-not incorporated,ACLSWKR_ State government,ACLSWKR_ Without pay,AHGA_ 10th grade,AHGA_ 11th grade,AHGA_ 12th grade no diploma,AHGA_ 1st 2nd 3rd or 4th grade,AHGA_ 5th or 6th grade,AHGA_ 7th and 8th grade,AHGA_ 9th grade,AHGA_ Associates degree-academic program,AHGA_ Associates degree-occup /vocational,AHGA_ Bachelors degree(BA AB BS),AHGA_ Children,AHGA_ Doctorate degree(PhD EdD),AHGA_ High school graduate,AHGA_ Less than 1st grade,AHGA_ Masters degree(MA MS MEng MEd MSW MBA),AHGA_ Prof school degree (MD DDS DVM LLB JD),AHGA_ Some college but no degree,AMARITL_ Divorced,AMARITL_ Married-A F spouse present,AMARITL_ Married-civilian spouse present,AMARITL_ Married-spouse absent,AMARITL_ Never married,AMARITL_ Separated,AMARITL_ Widowed,AMJIND_ Agriculture,...,HHDFMX_ Grandchild 18+ never marr not in subfamily,HHDFMX_ Grandchild 18+ spouse of subfamily RP,HHDFMX_ Grandchild <18 ever marr not in subfamily,HHDFMX_ Grandchild <18 never marr RP of subfamily,HHDFMX_ Grandchild <18 never marr child of subfamily RP,HHDFMX_ Grandchild <18 never marr not in subfamily,HHDFMX_ Householder,HHDFMX_ In group quarters,HHDFMX_ Nonfamily householder,HHDFMX_ Other Rel 18+ ever marr RP of subfamily,HHDFMX_ Other Rel 18+ ever marr not in subfamily,HHDFMX_ Other Rel 18+ never marr RP of subfamily,HHDFMX_ Other Rel 18+ never marr not in subfamily,HHDFMX_ Other Rel 18+ spouse of subfamily RP,HHDFMX_ Other Rel <18 ever marr RP of subfamily,HHDFMX_ Other Rel <18 ever marr not in subfamily,HHDFMX_ Other Rel <18 never marr child of subfamily RP,HHDFMX_ Other Rel <18 never marr not in subfamily,HHDFMX_ Other Rel <18 never married RP of subfamily,HHDFMX_ Other Rel <18 spouse of subfamily RP,HHDFMX_ RP of unrelated subfamily,HHDFMX_ Secondary individual,HHDFMX_ Spouse of RP of unrelated subfamily,HHDFMX_ Spouse of householder,HHDREL_ Child 18 or older,HHDREL_ Child under 18 ever married,HHDREL_ Child under 18 never married,HHDREL_ Group Quarters- Secondary individual,HHDREL_ Householder,HHDREL_ Nonrelative of householder,HHDREL_ Other relative of householder,HHDREL_ Spouse of householder,MIGSAME_ No,MIGSAME_ Not in universe under 1 year old,MIGSAME_ Yes,PARENT_ Both parents present,PARENT_ Father only present,PARENT_ Mother only present,PARENT_ Neither parent present,PARENT_ Not in universe
0,73,0,0,0,0,1700.09,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1
1,58,4,34,0,52,1053.55,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1
2,18,0,0,0,0,991.95,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1
3,9,0,0,0,0,1758.14,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0
4,10,0,0,0,0,1069.16,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0


In [0]:
train_target = train_data.pop('PTOTVAL')

In [0]:
test_url = 'https://github.com/PacktWorkshops/The-TensorFlow-Workshop/blob/master/Chapter06/dataset/census-income-test.csv?raw=true'

In [0]:
test_data = pd.read_csv(test_url, usecols=usecols)
test_data.head()

,AAGE,ADTIND,ADTOCC,SEOTR,WKSWORK,PTOTVAL,ACLSWKR_ Federal government,ACLSWKR_ Local government,ACLSWKR_ Never worked,ACLSWKR_ Not in universe,ACLSWKR_ Private,ACLSWKR_ Self-employed-incorporated,ACLSWKR_ Self-employed-not incorporated,ACLSWKR_ State government,ACLSWKR_ Without pay,AHGA_ 10th grade,AHGA_ 11th grade,AHGA_ 12th grade no diploma,AHGA_ 1st 2nd 3rd or 4th grade,AHGA_ 5th or 6th grade,AHGA_ 7th and 8th grade,AHGA_ 9th grade,AHGA_ Associates degree-academic program,AHGA_ Associates degree-occup /vocational,AHGA_ Bachelors degree(BA AB BS),AHGA_ Children,AHGA_ Doctorate degree(PhD EdD),AHGA_ High school graduate,AHGA_ Less than 1st grade,AHGA_ Masters degree(MA MS MEng MEd MSW MBA),AHGA_ Prof school degree (MD DDS DVM LLB JD),AHGA_ Some college but no degree,AMARITL_ Divorced,AMARITL_ Married-A F spouse present,AMARITL_ Married-civilian spouse present,AMARITL_ Married-spouse absent,AMARITL_ Never married,AMARITL_ Separated,AMARITL_ Widowed,AMJIND_ Agriculture,...,HHDFMX_ Grandchild 18+ never marr not in subfamily,HHDFMX_ Grandchild 18+ spouse of subfamily RP,HHDFMX_ Grandchild <18 ever marr not in subfamily,HHDFMX_ Grandchild <18 never marr RP of subfamily,HHDFMX_ Grandchild <18 never marr child of subfamily RP,HHDFMX_ Grandchild <18 never marr not in subfamily,HHDFMX_ Householder,HHDFMX_ In group quarters,HHDFMX_ Nonfamily householder,HHDFMX_ Other Rel 18+ ever marr RP of subfamily,HHDFMX_ Other Rel 18+ ever marr not in subfamily,HHDFMX_ Other Rel 18+ never marr RP of subfamily,HHDFMX_ Other Rel 18+ never marr not in subfamily,HHDFMX_ Other Rel 18+ spouse of subfamily RP,HHDFMX_ Other Rel <18 ever marr RP of subfamily,HHDFMX_ Other Rel <18 ever marr not in subfamily,HHDFMX_ Other Rel <18 never marr child of subfamily RP,HHDFMX_ Other Rel <18 never marr not in subfamily,HHDFMX_ Other Rel <18 never married RP of subfamily,HHDFMX_ Other Rel <18 spouse of subfamily RP,HHDFMX_ RP of unrelated subfamily,HHDFMX_ Secondary individual,HHDFMX_ Spouse of RP of unrelated subfamily,HHDFMX_ Spouse of householder,HHDREL_ Child 18 or older,HHDREL_ Child under 18 ever married,HHDREL_ Child under 18 never married,HHDREL_ Group Quarters- Secondary individual,HHDREL_ Householder,HHDREL_ Nonrelative of householder,HHDREL_ Other relative of householder,HHDREL_ Spouse of householder,MIGSAME_ No,MIGSAME_ Not in universe under 1 year old,MIGSAME_ Yes,PARENT_ Both parents present,PARENT_ Father only present,PARENT_ Mother only present,PARENT_ Neither parent present,PARENT_ Not in universe
0,38,6,36,0,12,1032.38,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1
1,44,37,12,0,26,1462.33,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1
2,2,0,0,0,0,1601.75,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0
3,35,29,3,2,52,1866.88,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,NaN,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1
4,49,4,34,0,50,1394.54,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1


In [0]:
test_target = test_data.pop('PTOTVAL')

In [0]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout

In [0]:
tf.random.set_seed(8)

In [0]:
def model_builder(hp):
  model = tf.keras.Sequential()

  hp_l2 = hp.Choice('l2', values = [0.1, 0.01, 0.001])
  hp_units = hp.Int('units', min_value=128, max_value=512, step=64)

  reg_fc1 = Dense(hp_units, input_shape=(5,), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l=hp_l2))
  reg_fc2 = Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l=hp_l2))
  reg_fc3 = Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l=hp_l2))
  reg_fc4 = Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l=hp_l2))
  reg_fc5 = Dense(1)
  
  model.add(reg_fc1)
  model.add(reg_fc2)
  model.add(reg_fc3)
  model.add(reg_fc4)
  model.add(reg_fc5)
  
  hp_learning_rate = hp.Choice('learning_rate', values = [0.01, 0.001]) 

  optimizer = tf.keras.optimizers.Adam(hp_learning_rate)
  model.compile(optimizer=optimizer, loss='mse', metrics=['mse'])
  
  return model

In [0]:
!pip install -q -U keras-tuner
import kerastuner as kt

     |████████████████████████████████| 61kB 1.8MB/s 


In [0]:
tuner = kt.BayesianOptimization(model_builder, objective = 'val_mse', max_trials = 10)

In [0]:
tuner.search(train_data, train_target, validation_data=(test_data, test_target))

6236/6236 [==============================] - 30s 5ms/step - loss: 1058954.7500 - mse: 1058954.0000 - val_loss: 1017614.1875 - val_mse: 1017613.6250


6236/6236 [==============================] - 48s 8ms/step - loss: 1052640.3750 - mse: 1052539.2500 - val_loss: 1029511.6875 - val_mse: 1029406.0625


6236/6236 [==============================] - 49s 8ms/step - loss: 1053191.8750 - mse: 1053180.3750 - val_loss: 1024192.7500 - val_mse: 1024180.8125


6236/6236 [==============================] - 31s 5ms/step - loss: 1057730.1250 - mse: 1057729.2500 - val_loss: 1018746.9375 - val_mse: 1018745.9375


6236/6236 [==============================] - 31s 5ms/step - loss: 1059136.3750 - mse: 1059135.5000 - val_loss: 1016917.3125 - val_mse: 1016916.5000


...

6236/6236 [==============================] - 30s 5ms/step - loss: 1059035.6250 - mse: 1059034.5000 - val_loss: 1018459.7500 - val_mse: 1018458.7500


6236/6236 [==============================] - 30s 5ms/step - loss: 1059626.3750 - mse: 1059625.6250 - val_loss: 1017563.4375 - val_mse: 1017562.7500


6236/6236 [==============================] - 30s 5ms/step - loss: 1059415.5000 - mse: 1059414.6250 - val_loss: 1016791.0000 - val_mse: 1016790.0000


6236/6236 [==============================] - 30s 5ms/step - loss: 1058323.2500 - mse: 1058322.3750 - val_loss: 1018271.4375 - val_mse: 1018270.5625


INFO:tensorflow:Oracle triggered exit


In [0]:
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

In [0]:
best_units = best_hps.get('units')
best_units

128

In [0]:
best_lr = best_hps.get('learning_rate')
best_lr

0.001

In [0]:
best_l2 = best_hps.get('l2')
best_l2

0.001

In [0]:
model = tuner.hypermodel.build(best_hps)

In [0]:
model.fit(train_data, train_target, epochs=5, validation_data=(test_data, test_target))

Epoch 1/5
6236/6236 [==============================] - 30s 5ms/step - loss: 1057504.6250 - mse: 1057503.7500 - val_loss: 1018363.1250 - val_mse: 1018362.1250
Epoch 2/5
6236/6236 [==============================] - 30s 5ms/step - loss: 1004640.6250 - mse: 1004639.8125 - val_loss: 999888.6250 - val_mse: 999887.9375
Epoch 3/5
6236/6236 [==============================] - 30s 5ms/step - loss: 1000052.0000 - mse: 1000051.3125 - val_loss: 995685.3750 - val_mse: 995684.6250
Epoch 4/5
6236/6236 [==============================] - 30s 5ms/step - loss: 996971.3125 - mse: 996970.6250 - val_loss: 1002984.3750 - val_mse: 1002983.8125
Epoch 5/5
6236/6236 [==============================] - 30s 5ms/step - loss: 994174.6875 - mse: 994174.0000 - val_loss: 989336.3125 - val_mse: 989335.4375
